![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **SentenceEntityResolverModel**

This notebook will cover the different parameters and usages of `SentenceEntityResolverModel`. This annotator maps clinical entities to a particilar ontology / curated dataset using sentence embeddings.

**📖 Learning Objectives:**

1. Understand the application and relevance of these models in healthcare data analysis

2. Map clinical entities to standard codes (ICD-10, RxNorm, SNOMED, etc.)

3. Become comfortable using the different parameters of the annotator.


**🔗 Helpful Links:**

- Documentation : [SentenceEntityResolverModel](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#sentenceentityresolver)

- Python Docs : [SentenceEntityResolverModel](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/resolution/sentence_entity_resolver/index.html)

- Scala Docs : [SentenceEntityResolverModel](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/finance/chunk_classification/resolution/SentenceEntityResolverModel.html)

- For extended examples of usage, see the [Spark NLP Workshop repository](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/healthcare-nlp).

## **🎬 Colab Setup**

In [1]:
!pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 715

In [2]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving spark_nlp_for_healthcare_spark_ocr_8734_v514.json to spark_nlp_for_healthcare_spark_ocr_8734_v514.json


In [3]:
from johnsnowlabs import nlp

nlp.install()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734_v514.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734_v514.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.1.4-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.1.4 installed! ✅ Heal the planet with NLP! 


In [4]:
from johnsnowlabs import nlp, medical
import pyspark.sql.functions as F
import pandas as pd

spark = nlp.start()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734_v514.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.4, 💊Spark-Healthcare==5.1.4, running on ⚡ PySpark==3.1.2


## **🖨️ Input/Output Annotation Types**

- Input: `SENTENCE_EMBEDDINGS`

- Output: `ENTITY`

## **🔎 Parameters**


- `distanceFunction`: Determines how the distance between different entities will be calculated. Either `COSINE` or `EUCLIDEAN`.

- `neighbours`: The number of neighbours to consider when computing the distances.

- `caseSensitive`: WWhether to consider text casing or not.

- `threshold`: Threshold of the distance between nodes to consider.


### `setDistanceFunction()`



Defines the distance function to use, either Euclidean or cosine.  

In [5]:
documentAssembler = (
    nlp.DocumentAssembler().setInputCol("text").setOutputCol("ner_chunk")
)

sbert_embedder = (
    nlp.BertSentenceEmbeddings.pretrained(
        "sbiobert_base_cased_mli", "en", "clinical/models"
    )
    .setInputCols(["ner_chunk"])
    .setOutputCol("sentence_embeddings")
    .setCaseSensitive(False)
)

rxnorm_resolver = (
    medical.SentenceEntityResolverModel.pretrained(
        "sbiobertresolve_rxnorm_augmented", "en", "clinical/models"
    )
    .setInputCols(["sentence_embeddings"])
    .setOutputCol("rxnorm_code")
    .setDistanceFunction("EUCLIDEAN")
)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[OK!]


In [6]:
text = 'metformin 100 mg'
df = spark.createDataFrame([[text]]).toDF("text")

In [7]:
results = rxnorm_pipelineModel.transform(df)

In [8]:
results.show(truncate=100)

+----------------+----------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|            text|                                                 ner_chunk|                                                                                 sentence_embeddings|                                                                                         rxnorm_code|
+----------------+----------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|metformin 100 mg|[{document, 0, 15, metformin 100 mg, {sentence -> 0}, []}]|[{sentence_embeddings, 0, 15, metformin 100 mg, {sentence -> 0, token -> metformin 

In [9]:
results.select("ner_chunk.result", "rxnorm_code.result").show()

+------------------+---------+
|            result|   result|
+------------------+---------+
|[metformin 100 mg]|[2200518]|
+------------------+---------+



In [10]:
rxnorm_resolver.setDistanceFunction("COSINE")
results = rxnorm_pipelineModel.transform(df)
results.select("ner_chunk.result", "rxnorm_code.result").show()

+------------------+--------+
|            result|  result|
+------------------+--------+
|[metformin 100 mg]|[858858]|
+------------------+--------+



### `setNeighbours`

The number of neighbours to find candidate codes, later filtered by the threshold value.

In [11]:
rxnorm_resolver.setNeighbours(5)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)
results = rxnorm_pipelineModel.transform(df)
results.select(
    F.explode(
        F.arrays_zip(
            results.rxnorm_code.result, results.rxnorm_code.metadata
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("rxnorm_code"),
    F.expr("cols['1']['all_k_results']").alias("all_k_results"),
    F.expr("cols['1']['all_k_resolutions']").alias("all_k_resolutions"),
).show(
    truncate=False
)


+-----------+---------------------------------+--------------------------------------------------------------------------------------+
|rxnorm_code|all_k_results                    |all_k_resolutions                                                                     |
+-----------+---------------------------------+--------------------------------------------------------------------------------------+
|858858     |858858:::198370:::316844:::437723|carnosine 100 MG:::torsemide 100 MG Oral Tablet:::torsemide 100 MG:::azintamide 100 MG|
+-----------+---------------------------------+--------------------------------------------------------------------------------------+



In [12]:
rxnorm_resolver.setNeighbours(50)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)
results = rxnorm_pipelineModel.transform(df)
results.select(
    F.explode(
        F.arrays_zip(
            results.rxnorm_code.result, results.rxnorm_code.metadata
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("rxnorm_code"),
    F.expr("cols['1']['all_k_results']").alias("all_k_results"),
    F.expr("cols['1']['all_k_resolutions']").alias("all_k_resolutions"),
).show(
    truncate=False
)


+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### `setCaseSensitive`

The models can be trained as case sensitive or not, so this parameter can have unexpected outcomes depending on the pretrained model used.

In [13]:
rxnorm_resolver.setCaseSensitive(False)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)
results = rxnorm_pipelineModel.transform(df)
results.select(
    F.explode(
        F.arrays_zip(
            results.rxnorm_code.result, results.rxnorm_code.metadata
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("rxnorm_code"),
    F.expr("cols['1']['all_k_results']").alias("all_k_results"),
    F.expr("cols['1']['all_k_resolutions']").alias("all_k_resolutions"),
).show(
    truncate=False
)


+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
rxnorm_resolver.setCaseSensitive(True)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)
results = rxnorm_pipelineModel.transform(df)
results.select(
    F.explode(
        F.arrays_zip(
            results.rxnorm_code.result, results.rxnorm_code.metadata
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("rxnorm_code"),
    F.expr("cols['1']['all_k_results']").alias("all_k_results"),
    F.expr("cols['1']['all_k_resolutions']").alias("all_k_resolutions"),
).show(
    truncate=False
)


+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### `setThreshold`

This parameter removes neighbors where the distance is greather than the threshold value. The value will be sensitive whether you are using cosine or Euclidean distance functions.

In [15]:
rxnorm_resolver.getThreshold()

1000.0

In [16]:
rxnorm_resolver.setDistanceFunction("EUCLIDEAN").setThreshold(500)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)
results = rxnorm_pipelineModel.transform(df)
results.select(
    F.explode(
        F.arrays_zip(
            results.rxnorm_code.result, results.rxnorm_code.metadata
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("rxnorm_code"),
    F.expr("cols['1']['all_k_results']").alias("all_k_results"),
    F.expr("cols['1']['all_k_resolutions']").alias("all_k_resolutions"),
).show(
    truncate=False
)


+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
rxnorm_resolver.setDistanceFunction("EUCLIDEAN").setThreshold(7)

rxnorm_pipelineModel = nlp.PipelineModel(
    stages=[documentAssembler, sbert_embedder, rxnorm_resolver]
)
results = rxnorm_pipelineModel.transform(df)
results.select(
    F.explode(
        F.arrays_zip(
            results.rxnorm_code.result, results.rxnorm_code.metadata
        )
    ).alias("cols")
).select(
    F.expr("cols['0']").alias("rxnorm_code"),
    F.expr("cols['1']['all_k_results']").alias("all_k_results"),
    F.expr("cols['1']['all_k_resolutions']").alias("all_k_resolutions"),
).show(
    truncate=False
)


+-----------+-------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rxnorm_code|all_k_results                                                                                    |all_k_resolutions                                                                                                                                                                                                                                                                                                                                       |
+-----------+-------------------------------------------------------------------------